In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

#import IPython.display as ipd
import numpy as np
import librosa
import librosa.display

import scipy
from scipy import signal
from pydub import AudioSegment
#from pydub.playback import play

from scipy.io import wavfile

import tensorflow as tf
import tensorflow.contrib.eager as tfe

tfe.enable_eager_execution()

# For CPU only
#pip install tf-nightly  
# For GPU support
#pip install tf-nightly-gpu

import os
import shutil
import random

Instructions for updating:
Use the retry module or similar alternatives.


# validata set 생성

In [83]:
train_path = "C:\\Users\\xp102\\Desktop\\투빅스 오디오\\audio_word"

train_ratio = 0.7
val_ratio = 1 - train_ratio

classnames = os.listdir(train_path)
if '_background_noise_' in classnames:
    classnames.remove('_background_noise_')

In [84]:
# 화자별로 따로 나눌 new_train, val_train폴더 및 클래스 폴더 생성
new_train_path = "C:\\Users\\xp102\\Desktop\\투빅스 오디오\\train_audio"
val_train_path  = "C:\\Users\\xp102\\Desktop\\투빅스 오디오\\val_audio"

In [99]:
os.mkdir(new_train_path)
os.mkdir(val_train_path)

for m in classnames:
    target = os.path.join(new_train_path,m)
    os.mkdir(target)
    
    target = os.path.join(val_train_path,m)
    os.mkdir(target) 

In [102]:
for d in classnames:
    print(d)
    temp = os.listdir(os.path.join(train_path,d))
    temp2 = [] 

    for i in temp:
        temp2.append(i.split("_")[0])  # 화자에 대한 정보

    name_set = set(temp2) # unique

    train_num = round(len(name_set) * train_ratio)
    val_num = round(len(name_set) * val_ratio)
    train_name = []
    val_name = []

    values = np.array(temp2)

    for j in range(train_num):
        searchval = list(name_set)[j]
        ii = np.where(values == searchval)[0]
        train_name.append( [temp[x] for x in list(ii)] )
        
    train_name2 = [y for x in train_name for y in x] 

    for q in train_name2:
        shutil.copy(src = os.path.join(train_path, d,q),  #파일 복사
                   dst = os.path.join(new_train_path, d,q))


    for k in range(train_num,len(name_set)):
        searchval = list(name_set)[k]
        ii = np.where(values == searchval)[0]
        val_name.append( [temp[x] for x in list(ii)] )
        
    val_name2 = [y for x in val_name for y in x]
        
    for q in val_name2:
        shutil.copy(src = os.path.join(train_path, d,q),  #파일 복사
                    dst = os.path.join(val_train_path, d,q))

계단
과제
구름



# train, val, test 고화질 스펙트로그램 이미지 생성

In [108]:
train_image_path = "C:\\Users\\xp102\\Desktop\\투빅스 오디오\\train_image"
val_image_path  = "C:\\Users\\xp102\\Desktop\\투빅스 오디오\\val_image"

os.mkdir(train_image_path)
os.mkdir(val_image_path)
for m in classnames:
    target = os.path.join(train_image_path,m)
    os.mkdir(target)
    
    target = os.path.join(val_image_path,m)
    os.mkdir(target) 

In [104]:
def hz_to_mel(freq):
    return 1127. * tf.log(1.0 + (freq / 700.))

def mel_to_hz(mel):
    return 700.*(tf.exp(mel/1127.)-1.)

def multi_ffts_to_mel(freq_array, n_mels=128):
    melfreq_array = tf.expand_dims(hz_to_mel(freq_array),0)
  
    mel_edges = tf.lin_space(hz_to_mel(tf.reduce_min(freq_array)), #or just use 0
                           hz_to_mel(tf.reduce_max(freq_array)), #or SR/2
                           n_mels+2)
  
    lower_edge_mel, center_mel, upper_edge_mel =tf.split(tf.contrib.signal.frame(mel_edges, 3, 1, axis=-1), 3, axis=-1)

    wt_down = (melfreq_array - lower_edge_mel) / (center_mel - lower_edge_mel)
    wt_up = (upper_edge_mel - melfreq_array) / (upper_edge_mel - center_mel)
  
    mel_weights_matrix = tf.maximum(0.0, tf.minimum(wt_down, wt_up))
    center_mel_freqs = mel_to_hz(center_mel) 
  
    return mel_weights_matrix, center_mel_freqs

def audioframes2logmelspec(b_framed_signal, n_ffts=5, wvls_per_window_hinge=16, n_mel=128, fft_l1=1024, sr=50000):
  # batch_framed_signal has shape: (batch_size x n_windows x fft_l1)
  # decrease weights for samples w/ more than wvls_per_window_hinge
  # wvls_per_window_hinge method could be improved, maybe weight~pmf of poisson?
    
    fft1_space = tf.lin_space(0., .5, 1+fft_l1//2)[1:]
    freq_list =[sr*fft1_space] 
    n_wv_list =[fft_l1*fft1_space]

    fft_list =[tf.spectral.rfft(b_framed_signal)[:,:,1:]]
  
    for i in range(1,n_ffts):
        fft_lnew = fft_l1//2**i
        fftnew_space = tf.lin_space(0., .5, 1+fft_lnew//2)[1:]
    
        freq_list.append(sr*fftnew_space)
        n_wv_list.append(fft_lnew*fftnew_space)
    
        frames_new = b_framed_signal[:, :, (fft_l1-fft_lnew)//2:(fft_l1-fft_lnew)//2+fft_lnew]
        fft_list.append(tf.spectral.rfft(frames_new)[:,:,1:])
    
  
    freq_concat = tf.concat(freq_list, axis=-1)
    n_wv_concat = tf.concat(n_wv_list, axis=-1)
    fft_concat = tf.concat(fft_list, axis=-1)
    
    magnitude_spectros = tf.abs(fft_concat)

    mel_wts, center_mel_freqs = multi_ffts_to_mel(freq_concat, n_mel)
    wvls_wts = tf.where(n_wv_concat>wvls_per_window_hinge, wvls_per_window_hinge/n_wv_concat, tf.ones_like(n_wv_concat))
  
    mel_spectro=tf.tensordot(magnitude_spectros, (mel_wts*tf.expand_dims(wvls_wts,0)),axes = [[2], [1]])

    log_mel_spectro = tf.log(mel_spectro+1e-7)
  
    return tf.expand_dims(log_mel_spectro, -1), center_mel_freqs

In [105]:
def plot_several_logmelspec(path,save_dir):
    fig = plt.figure(figsize=(5, 5))

    sr, y = wavfile.read(path)
    signal = y.astype(np.float32) / np.iinfo(np.int16).max

    b_signals = tf.expand_dims(signal, axis=0)

    b_framed_signal = tf.contrib.signal.frame(b_signals, frame_length=1024, frame_step = 32)
    log_mel_spectro, center_mel_freqs = audioframes2logmelspec(b_framed_signal, sr=sr)

    librosa.display.specshow(log_mel_spectro[0,:,:,0].numpy().T, sr=sr, 
                             #x_axis='time', y_axis='mel', 
                             hop_length=32, 
                             fmin=tf.reduce_min(center_mel_freqs), 
                             fmax=tf.reduce_max(center_mel_freqs), 
                             cmap='coolwarm')

    fig.savefig(save_dir, bbox_inches='tight', pad_inches=0)
    #plt.tight_layout()
    plt.close('all') 

# make train, test Image 

In [109]:
for name in classnames:
    print(name)
    path = os.path.join(new_train_path,name)
    audio_list = os.listdir(path)
    
    for audio in audio_list:
        audio_dir = os.path.join(path,audio)
        save_dir = os.path.join(train_image_path,name,audio.split(".")[0]+".png")
        
        plot_several_logmelspec(audio_dir,save_dir)
        plt.close('all') # 너무 많이 쌓이면 에러가 난다.

계단
과제
구름


In [110]:
for name in classnames:
    print(name)
    path = os.path.join(val_train_path,name)
    audio_list = os.listdir(path)
    
    for audio in audio_list:
        audio_dir = os.path.join(path,audio)
        save_dir = os.path.join(val_image_path,name,audio.split(".")[0]+".png")
        
        plot_several_logmelspec(audio_dir,save_dir)
        plt.close('all') # 너무 많이 쌓이면 에러가 난다.

계단
과제
구름
